<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 4: Distillation and Retrieval Augmented Generation</h2>
<h4>Lecture: Dr. Guokan Shang<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, November 18, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>Novemver 23
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

# Install Requirements

In [ ]:
# Install required dependencies
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate vllm
!pip install jedi
# !pip -q install datasets==2.21.0 pandas==2.2.2
# !pip -q install chromadb==0.4.22
# !pip -q install "numpy<2.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/

# Part 1 - Model Distillation

> In this section, you’ll learn the difference between **white-box** and **black-box** distillation, generate **synthetic data** to train a **student model**, and implement **white-box distillation** to specialize a model for a **RAG on Wikipedia** use case.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:</br>
Explain briefly the difference between black-box and white-box distillation? </br> What are the advantages and inconvenients of each approach?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>

In White-Box Knowledge Distillation, the Student model $S$ has complete access to the Teacher model $T$, including its parameters $\theta$, gradients, and intermediate feature maps. This transparency allows for feature-level distillation, where the objective is to force $S$ to mimic the internal activation maps of $T$, ensuring the Student effectively "thinks" like the Teacher. While this method offers significant advantages such as higher performance, faster convergence, and better generalization, it has notable downsides: it poses privacy and security risks by exposing proprietary model weights and often necessitates a strict architectural dependency between $S$ and $T$.

Black-Box Knowledge Distillation, in contrast, operates under the constraint that the Teacher model $T$ is inaccessible to the Student. Without access to internal weights or features, $S$ must learn solely from the input data and the final output logits (or "soft" labels) generated by $T$. This approach results in lower information density and reduced data efficiency compared to white-box methods, as the rich intermediate supervisory signals are missing. However, it allows for greater flexibility in model architecture and better preserves the privacy of the Teacher model.

<hr style="border:10px solid green"> </hr>
</font></h4>


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 2:</br>
What is the main requirement for a teacher/student pair of models to perform white-box distillation?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
Your answer here.

To perform white-box distillation, the system requires a specific structural alignment between the two models to allow for the transfer of internal information, not just the final output. This is most easily achieved if the teacher and student share an identical architecture, allowing for a direct one-to-one mapping of layers. However, in practical scenarios where the student is smaller, block projection layers are required. These layers serve as a bridge, projecting the student’s internal feature maps to match the dimensions of the teacher's corresponding blocks, ensuring that the intermediate representations can be directly compared and optimized.
<hr style="border:10px solid green"> </hr>
</font></h4>


## 1.1 - Synthetic Data Generation

We're going to specialize a small 0.5B parameter model to perform RAG by distilling the abilities of a 7B parameter one.

For that we'll be using `Qwen/Qwen2.5-0.5B-Instruct` as student and `Qwen/Qwen2.5-7B-Instruct-AWQ` (quantized version of `Qwen2.5-7B-Instruct`) as teacher.  

In order to perform white-box distillation on generated answers we have two choices.

1. We can perform a forward pass with the teacher, a forward pass with student on the complete sequence, and backprop difference of logprobs using KL Loss.
2. Generation of samples with the teacher, save the logprobs and perform finetuning in a second step.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 3:</br>
What are the computational advantages of 1. vs 2.?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>

The first approach relies on a synchronous mechanism where both the Teacher and Student models are loaded into VRAM simultaneously. In this setup, a forward pass is executed on both models for every batch to compute the divergence between their distributions, and backpropagation is applied solely to the Student. This process utilizes the loss function $$\mathcal{L}_{KD} = \mathcal{L}_{KL}(P_{teacher} || P_{student})$$
The primary computational advantage of this method is the elimination of storage overhead, as there is no need to save massive amounts of log-probabilities to disk. Furthermore, it allows for dynamic adaptation, ensuring the student learns from the teacher's exact distribution during every step of the training process.

In contrast, the second approach separates the workflow into a distinct generation phase followed by fine-tuning. This involves performing a large-scale inference run with the Teacher to save the generated tokens and associated log-probabilities (typically Top-K) into a static dataset. This method offers significant efficiency gains because the inference phase can utilize highly optimized engines like vLLM or SGLang, which are considerably faster than a standard training loop forward pass. Additionally, this decoupling provides flexibility in scaling; the teacher's generation can be distributed across different hardware, and the student's training can be restarted with new hyperparameters without the computational cost of re-running the teacher.
<hr style="border:10px solid green"> </hr>
</font></h4>

We're going to generate a bunch of questions related to wikipedia paragraphs.
For that we need to establish a system prompt that will allow for easy extraction.

In [ ]:
system_prompt = """
You are a question generator.
The user will provide:

```json
{"title": "the title of an article", "paragraph": "a paragraph from that article"}
```

Your task:

* Generate one clear, self-contained question that can be answered using only the provided paragraph.
* The question must be **specific**, **unambiguous**, and directly tied to the paragraph’s content.
* Return the result with the question as a valid JSON** in the form:

```json
{
  "question": "your question here"
}
```

Example:
User input:

```json
{
"title": "The Moon Landing",
"paragraph": "On July 20, 1969, Neil Armstrong became the first human to set foot on the Moon, followed by Buzz Aldrin."
}
```
Assistant output:

```json
{
  "question": "Who was the first human to set foot on the Moon during the Apollo 11 mission?"
}
```
"""

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 4:</br>
In this system prompt, we don't generate answers, only questions. Explain why it's necessary in the context of white-box distillation.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>

The primary necessity for separating question generation from answer generation lies in aligning the training process with the specific structure of the Retrieval-Augmented Generation (RAG) inference flow. In a production setting, the student model functions as a conditional generator that receives a fixed question and context as input and must produce a corresponding answer. Consequently, the distillation objective is to train the student to mimic the teacher's conditional probability distribution, denoted as $P(\text{Answer} | \text{Question})$. By isolating the question as a pre-determined input during the data generation phase, we ensure that the student learns to respond to external queries rather than simply continuing a text sequence.

From a probabilistic standpoint, generating the question and answer in a single pass would fundamentally alter the learning objective. If the teacher were prompted to produce a Q\&A pair simultaneously, it would be modeling the joint probability $P(\text{Question}, \text{Answer} | \text{Context})$. In this scenario, the resulting logits would partially encode the probability of constructing the question text itself, rather than focusing purely on the reasoning required to derive the answer.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 5:</br>
For synthetic data generation we'll be using vLLM.
vLLM is an optimized llm inference engine that can improve generation speed thanks to hardware specific optimization and computational tricks such as Prefix KV Caching.
(https://docs.vllm.ai/en/latest/features/automatic_prefix_caching.html)</br></br> 1. Explain why prefix caching will be very efficient in our case? </br></br>
2. What sampling `temperature` should we use? Justify.



<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>


1.Prefix caching is particularly efficient in this scenario because the prompts used for synthetic data generation share a significant amount of static content. In the context of generating questions or answers from Wikipedia paragraphs, the system instructions and the reference documents themselves serve as a common prefix across multiple distinct requests. vLLM's Automatic Prefix Caching mechanism detects when the beginning of a new prompt matches a sequence that has already been processed. Instead of recomputing the attention mechanism for these repeated tokens, the engine reuses the pre-computed Key-Value (KV) cache stored in memory.  This capability is crucial for our workflow because it eliminates redundant computation on the heavy context window, allowing the inference engine to focus its computational resources solely on generating the unique components of the response, thereby drastically improving throughput.

2.For this distillation task, we should utilize a low sampling temperature. The objective of generating synthetic data for RAG is to create a dataset that is factually accurate and strictly aligned with the information contained in the source documents. A lower temperature reduces the randomness in the model's token selection process, forcing the distribution to peak around the most probable, logical continuations. This determinism is essential to minimize hallucinations or creative deviations that could introduce noise into the student model's training data. By keeping the temperature low, we ensure that the teacher model functions as a faithful reasoner that adheres to the provided context rather than a creative generator.

<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Optimized Generation of Synthetic Questions

Complete the below code with the adequate options (prefix caching and `temperature`)

<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
from vllm import LLM
from vllm import SamplingParams
import os
path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"
llm = LLM(model=path_teacher, quantization="awq", dtype="float16", tensor_parallel_size=1, enable_prefix_caching=True, gpu_memory_utilization=0.6, max_model_len=2048, enforce_eager=True)  # To Complete
sampling_params = SamplingParams(temperature = 0.1, max_tokens=400) # To Complete


INFO 11-21 12:54:58 [utils.py:253] non-default args: {'dtype': 'float16', 'max_model_len': 2048, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'quantization': 'awq', 'enforce_eager': True, 'model': 'Qwen/Qwen2.5-7B-Instruct-AWQ'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

INFO 11-21 12:55:20 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-21 12:55:20 [model.py:1745] Using max model len 2048
INFO 11-21 12:55:24 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 11-21 12:55:28 [system_utils.py:103] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-21 13:02:15 [llm.py:352] Supported tasks: ['generate']


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Question Generation</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

We're going to use the llm.chat vLLM api to generate our samples.
Make the adequate code to generate the data and save it in a questions.jsonl file.

Entries of jsonl file should look like:
```json
{
  "id_doc": <wikipedia_article_id>,
  "id_paragraph": <paragraph_dataset_id>,
  "question": <generated_question>,
  "title": <title_wikipedia_article>,
  "paragraph": <text_of_paragraph>
}
```

You should:

1. Complete `def extract_question(generated_text: str) -> str:` to extract the generated question.
2. Complete `def conversation_generator` to output a generator directly ingestible by `llm.chat` api
3. Complete the dataloader and for loop to generate samples by batches of 4
4. Save the generated questions in a `questions.jsonl` file

In [ ]:
from typing import Iterable, Iterator, List
import json
def extract_question(generated_text: str) -> str:

    """Extract the question from the generated text. If the question is not\
    following the right format return None."""

    try:
      j: dict = json.loads(generated_text)

      assert "question" in j
      return j["question"]
    except AssertionError:
      return None


def conversation_generator(
    entries: Iterable[dict],
    system_prompt: str
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    for entry in entries:
      conversation: List[dict]= [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": (
            f"Title: {entry['title']}\n"
            f"Paragraph: {entry['paragraph']}"
        )}
        ]
    yield {
            "messages": conversation,
            "metadata": entry
        }

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset

ds = load_dataset("EvanD/Lab4_wikiparagraphs")

batch_size = 4
system_prompt = (
    "You are an expert at generating reading comprehension questions. "
    "Given a title and a paragraph, generate a single relevant question based on the text. "
    "Output ONLY a JSON object with the format: {\"question\": \"Your generated question here\"}."
)

conversations = list(conversation_generator(ds['train'], system_prompt)) # To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)


In [ ]:
from tqdm.auto import tqdm
import jsonlines


with jsonlines.open("questions.jsonl", "w") as writer:
    for i, batch in tqdm(enumerate(dataloader)):
        id_paragraph = i * batch_size
        llm_inputs = [item["messages"] for item in batch]
        outputs = llm.chat(llm_inputs,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for j, output in enumerate(outputs):
          entry = {}
          #print(output)
          q = extract_question(output.outputs[0].text)

          # You must verify that the output actually contains a question and write it
          # To Complete
          current_paragraph_id = (i * batch_size) + j
          original_data = batch[j]["metadata"]
          if q is not None:
                entry = {
                    #"id_doc": dataloader.get('id', None), # Assuming 'id' exists in source dataset
                    "id_paragraph": current_paragraph_id,
                    "question": q,
                    "title": original_data['title'],
                    "paragraph": original_data['paragraph']
                }
                writer.write(entry)


0it [00:00, ?it/s]

## 1.2 - Logprobs Generation

We get to the second part of this distillation where we are interested in distilling the answers logprobs of our teacher model to specialize our 0.5B model to perform Retrieval Augmented Generation (RAG).

First we're going to generate the logprobs with our 7B parameter model.

We'll use the following system prompt:

In [ ]:
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: Complete Conversation Generator</br>

<hr style="border:10px solid blue"> </hr>"
</font></h4>

Based on the previous code you made to generate questions, make a second one to generate answers and saving logprobs. We updated the sampling parameters of vLLM to return the logprobs and the token ids of the 20 most probable tokens.

As this can result in high quantity of data in practice we're going to store generated conversations in a .jsonl file and generated logprobs and token ids to a hdf5 file (see https://docs.h5py.org/en/stable/ for more information on hdf5).

`save_logprobs_hdf5()` is already implemented for you, and allows to save the logprobs to a .h5 hdf5 file, and increments sequences automatically

The structure of the conversation generator `conversation_generator()` function is implemented, you need to complete it.



In [ ]:
import h5py, os
import numpy as np

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs  = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs",  data=logprobs,  compression="gzip")





<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 6:</br>
When completing conversation_generator we have several ways of sampling.
What are good paragraph sampling strategies we could use to ensure good performance of downstream model?

<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>


To ensure high-quality training data, we can employ Length-Based Sampling, which filters paragraphs by token count (e.g., 50 to 512) to prevent context starvation or truncation issues. Additionally, Semantic Diversity Sampling uses embeddings and clustering (like K-Means)  to select data proportionally from different conceptual groups, ensuring broad coverage and minimizing redundancy. Finally, Entity Density Sampling prioritizes paragraphs with high counts of Named Entities; this strategy forces the model to focus on factual density, thereby reducing hallucinations by grounding the training in concrete details.

<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)  # We add the title to contextualize the paragraph

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""
    all_doc_ids = list(doc_id_to_paragraphs.keys())

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]] # We add the title to contextualize the paragraph

        # Try to get the doc ID if available to find siblings
        current_doc_id = q_p.get("id_doc")

        while len(paragraphs) < number_of_paragraphs_in_context:
          candidate = None

          # With 50% probability, add a paragraph from the same article
          if current_doc_id in doc_id_to_paragraphs and random.random() < 0.5:
              possible_paragraphs = doc_id_to_paragraphs[current_doc_id]
              temp_candidate = random.choice(possible_paragraphs)
              if temp_candidate not in paragraphs:
                  candidate = temp_candidate

          # Add a paragraph from a different article (fallback or if probability > 0.5)
          if candidate is None:
              random_id = random.choice(all_doc_ids)
              # Ensure we don't pick the same document
              while random_id == current_doc_id:
                  random_id = random.choice(all_doc_ids)

              temp_candidate = random.choice(doc_id_to_paragraphs[random_id])
              if temp_candidate not in paragraphs:
                  candidate = temp_candidate

          # Append the valid candidate
          if candidate:
              paragraphs.append(candidate)

        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q_p["question"]}
        ]
        yield conversation



In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Iterable


batch_size = 4

conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt)) # To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

sampling_params = SamplingParams(temperature=0.2, max_tokens=400, logprobs=20)

with jsonlines.open("conversations_rag.jsonl", "w") as writer:
    for batch in tqdm(dataloader):
        out = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for i, b in enumerate(batch):
            text = out[i].outputs[0].text
            b.append({"role": "assistant", "content": text})
            writer.write(b)
        save_logprobs_hdf5("logprobs.h5", [out[i].outputs[0].logprobs for i in range(len(out))])

'''
conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))
'''

  0%|          | 0/1 [00:00<?, ?it/s]

'\nconversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))\n'

## 1.3 - KL-Divergence and Distillation

**You should restart the notebook kernel to free the gpu memory from the 7B model that is no longer needed**

Now that we have the generated conversations and their logprobs we can train our 0.5B model to output the same distribution.

The attentive student would have noticed that we have an incomplete representation of the probability distribution over tokens due to only keeping the top-20 logprobs.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 7:</br>
What are the two solutions you can see to approximate full distillation despite only having the top-20 logprobs?

<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>


The first solution is Truncated KL Divergence, which focuses on capturing the relative relationships between the high-probability tokens, often referred to as "dark knowledge." Since the full distribution is unavailable, we re-normalize the teacher's top-20 probabilities so that they sum to one, and apply the same normalization to the student's corresponding logits. By computing the KL Divergence exclusively on this subset, the student learns the teacher's subtle preferences among plausible answers without being penalized for the missing tail of the distribution.

The second solution is Tail Mass Matching, which addresses the model's uncertainty by accounting for the probability mass of the unobserved tokens. This method involves calculating the sum of the probabilities for the teacher's top-20 tokens and adding a loss term that forces the student's aggregate probability for those same tokens to match this value. This ensures that the student mimics the teacher's confidence level; if the teacher spreads significant mass across the missing tail, the student is implicitly forced to reserve equivalent probability mass for the rest of the vocabulary, preventing overconfidence.

<hr style="border:10px solid green"> </hr>
</font></h4>

We supply two functions to help in this implementation:

- `find_subsequence()`that allows to find all the occurences of a token subsequence in a 1-D tensor
- `get_labels()` that allows to expand the top-20 logprobs to the whole vocabulary, implicitly setting prob to zero for other tokens
- `QwenKLDataset`that loads samples from .h5 and .jsonl files, remove problematic inconsistent tokenized examples and outputs samples tokenized for training.

To simplify we will train with a batch size of 1, you can implement gradient accumulation if you wish.

The `PREFIX_ASSISTANT`variable contains the tokens that encode for

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [2]:
import torch

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""

    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

In [ ]:
from torch.utils.data import Dataset
import jsonlines
import h5py


PREFIX_ASSISTANT = [198, 151644, 77091, 198]

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:

        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()

        return entry

KL-Divergence is a metric often used to quantify the difference of probability mass between two distributions.
It's not mathematically defined as a distance because of its asymetric nature:

$$D_{KL}(P \parallel Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)}$$

In knowledge distillation, we often minimize the **Kullback–Leibler divergence** between the teacher’s output distribution $P_T$ and the student’s output distribution $P_S$.

$$
D_{KL}(P_T \parallel P_S) = \sum_x P_T(x) \log \frac{P_T(x)}{P_S(x)}
$$
$$
D_{KL}(P_S \parallel P_T) = \sum_x P_S(x) \log \frac{P_S(x)}{P_T(x)}
$$


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 8:</br>
What qualitative difference would it make if we minimize $D_{KL}(P_T \parallel P_S)$ instead of $D_{KL}(P_S \parallel P_T)$? How would it affect the convergence of the student distribution?

<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 8: </b><br>

The direction of the KL Divergence fundamentally alters the convergence behavior of the student model. Minimizing the Forward KL Divergence, $D_{KL}(P_T \parallel P_S)$, which corresponds to standard Knowledge Distillation, results in "Mean-Seeking" or "Zero-Avoiding" behavior. Since the loss is weighted by the teacher's probability $P_T(x)$, the student is heavily penalized if it assigns zero probability to any region where the teacher has high probability. Consequently, the student stretches its distribution to cover all the teacher's modes, often resulting in a flatter, more inclusive distribution that averages out complex, multi-modal answers.

Conversely, minimizing the Reverse KL Divergence, $D_{KL}(P_S \parallel P_T)$, leads to "Mode-Seeking" or "Zero-Forcing" behavior. In this formulation, the loss is weighted by the student's probability $P_S(x)$, meaning the student incurs no penalty for ignoring valid regions of the teacher's distribution as long as it does not assign probability to regions where the teacher has none. This causes the student to "collapse" onto a single mode, effectively sharpening its output to focus entirely on one specific answer while ignoring other valid possibilities to minimize risk.
<hr style="border:10px solid green"> </hr>
</font></h4>

**You need to ensure you restarted the kernel and have sufficiently free memory (no 7B model running)**

check with `! nvidia-smi`

In [4]:
! nvidia-smi

Sun Nov 23 11:54:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----




This code is very basic, and allows you to test a training over the small number of samples you've generated. Due to limitations of Google Colab we cannot go much further, but this first part should have given you the basics on how to perform white-box distillation.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: Complete The Training Code</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>


- Loss function
- gradient accumulation handling



In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np


student_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct").cuda()
dataset = QwenKLDataset("logprobs.h5", "conversations_rag.jsonl") # To Complete

num_epochs = 2
grad_accum_steps = 8
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])
num_training_steps = num_epochs * len(loader) // grad_accum_steps
scheduler = get_linear_schedule_with_warmup(optimizer, 0, num_training_steps)
loss_fn = torch.nn.KLDivLoss(reduction="batchmean", log_target=True) # To Complete

for epoch in range(num_epochs):
    for i, batch in enumerate(loader):
        outputs = student_model(batch["inputs"]["input_ids"])
        start_idx = batch["idx_subseq"]
        seq_len = batch["seq_len"]

        student_logits = outputs.logits[:, start_idx-1 : start_idx-1+seq_len, :]


        student_logprobs = F.log_softmax(student_logits, dim=-1)

        target_logprobs = batch["labels"].unsqueeze(0)
        batch_loss = loss_fn(student_logprobs, target_logprobs)

        (batch_loss / grad_accum_steps).backward()
        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print (f"Epoch {epoch+1}, step {i+1}/{len(loader)}: loss = {batch_loss.item():.4f}")
    print(f"Epoch {epoch+1}: loss = {outputs.loss.item():.4f}")

student_model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

NameError: name 'QwenKLDataset' is not defined

# Part 2 - Retrieval Augmented Generation (RAG)

In this section, we will discuss the concept of **Retrieval-Augmented Generation (RAG)** — a framework that combines **information retrieval** and **language generation**. RAG enables language models to access **external knowledge sources** at inference time, reducing hallucinations and improving factual accuracy.

We will explore how to:
- Build and index a **Vector database** from a corpus (here: Wikipedia sample).
- Retrieve the most relevant documents given a query using **embedding-based similarity**.
- Integrate retrieval results into the **generation pipeline** to produce context-aware answers.


In [4]:
# restart the session and run
!pip -q install chromadb==0.4.22
!pip -q install "numpy<2.0" --force-reinstall
!pip -q install datasets==2.21.0 pandas==2.2.2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incom

In [5]:
import os
import json
import random
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel

In [6]:
# --- Configuration ---
SEED = 42
NUM_ROWS = 1000
DATA_PATH = "wikipedia_20231101_en_1000.csv"

if os.path.exists(DATA_PATH):
    print(f"✅ Found existing dataset at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    print("⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...")
    random.seed(SEED)

    # Load the Wikipedia dataset
    stream_ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True
    )

    buffered_stream = stream_ds.shuffle(seed=SEED, buffer_size=200_000)

    sampled = []
    for ex in buffered_stream:
        try:
            if int(ex["id"]) % 2 == 0:
                sampled.append(ex)
            if len(sampled) >= NUM_ROWS:
                break
        except:
            continue

    # Create DataFrame
    df = pd.DataFrame(sampled)[["id", "url", "title", "text"]]
    df.to_csv(DATA_PATH, index=False)
    print(f"💾 Dataset saved to {DATA_PATH}")


⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

💾 Dataset saved to wikipedia_20231101_en_1000.csv


In [7]:
#Display Basic Info ---
print("Sampled shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

Sampled shape: (1000, 4)
Columns: ['id', 'url', 'title', 'text']


,id,url,title,text
0,64741026,https://en.wikipedia.org/wiki/Indrani%20Perera,Indrani Perera,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,66847582,https://en.wikipedia.org/wiki/August%20Laur,August Laur,August Laur (9 October 1886 Vana-Põltsamaa Par...
2,66467526,https://en.wikipedia.org/wiki/Daniele%20Solcia,Daniele Solcia,Daniele Solcia (born 7 March 2001) is an Itali...
3,65913988,https://en.wikipedia.org/wiki/Eric%20Takabatake,Eric Takabatake,Eric Takabatake (born 9 January 1991) is a Bra...
4,64723650,https://en.wikipedia.org/wiki/Nafissath%20Radji,Nafissath Radji,Nafissath Radji (born 2 August 2002 in Porto-N...


In [8]:
# Visualize Random Wikipedia Articles ---

NUM_EXAMPLES = 3  # number of random samples to show
samples = df.sample(NUM_EXAMPLES, random_state=random.randint(0, 10000))

for _, row in samples.iterrows():
    display(HTML(f"""
    <hr style="border:2px solid #ccc">
    <h3><b>Title:</b> {row['title']}</h3>
    <p><b>URL:</b> <a href="{row['url']}" target="_blank">{row['url']}</a></p>
    <p style="text-align: justify;"><b>Text:</b><br>{row['text']}</p>
    """))


### **Document Chunking**

The first step in building a RAG pipeline is **chunking**, where large documents are divided into smaller, semantically coherent pieces.  
Chunking allows the retriever to work on manageable text segments instead of entire documents, improving retrieval precision and reducing computational load.  


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Naive Fixed-Length Chunking  
Split each document into overlapping fixed-length chunks to prepare text for retrieval.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [9]:

def text_splitting(text, chunk_length=300, chunk_overlap=100):
    """
    Splits text into fixed-length chunks with overlap.
    """
    out = []
    stride = chunk_length - chunk_overlap ## FILL THE GAP: define the stride as the effective step between chunks
    for i in range(0, len(text), stride):
        chunk = text[i : i + chunk_length] ## FILL THE GAP: extract a substring of size 'chunk_length' starting at 'i'
        out.append(chunk)
    return out

# Apply to all documents
df["naive_chunks"] = df["text"].apply(lambda t: text_splitting(t, chunk_length=300, chunk_overlap=100))

num_chunks = df["naive_chunks"].apply(len)
print(f"Average number of chunks per document: {num_chunks.mean():.2f}")
print(f"Total number of chunks: {num_chunks.sum()}")

example_idx = 0
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]["title"])
print("Original length:", len(df.iloc[example_idx]["text"]))
print("Number of chunks:", len(df.iloc[example_idx]["naive_chunks"]))
print("\nFirst 2 chunks:\n")
for i, c in enumerate(df.iloc[example_idx]["naive_chunks"][:2]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

Average number of chunks per document: 11.50
Total number of chunks: 11498

--- Example document ---
Title: Indrani Perera
Original length: 3012
Number of chunks: 16

First 2 chunks:

Chunk 1:
Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer. Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls 
--------------------------------------------------------------------------------
Chunk 2:
la Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls in the family. Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian. She has one elder sister, Mallika and one younger sister, Iranganie. Indrani studied at  Presbyteri
--------------------------------------------------------------------------------


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Paragraph-Aware Chunking  
Implement a smarter chunking strategy by using the ('.') as a boundary to split text into sentences or short paragraphs, then regroup them until reaching the desired chunk length.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [10]:
def text_splitting_paragraph(text, chunk_length=300):
    """
    Splits text by sentences/paragraphs (using '.' as boundary)
    and groups them until reaching the desired chunk length.
    """
    out = []
    paragraph_list = text.split('.') ## FILL THE GAP: split the text into smaller parts using '.' as a separator
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += par + "." ## FILL THE GAP: extend the ongoing chunk with the next segment
            length += len(par) + 1 ## FILL THE GAP: increment the total length accordingly
        else:
            if len(current_text) != 0:
                out.append(current_text)## FILL THE GAP: store the current completed chunk before starting a new one)
            current_text = par + "." ## FILL THE GAP: initialize a new chunk with the current paragraph
            length = len(par) + 1## FILL THE GAP: reset the chunk length counter
    if len(current_text) > 0:
        out.append(current_text)## FILL THE GAP: add the last remaining chunk to the list)
    return out


# Apply to all documents
df["paragraph_chunks"] = df["text"].apply(lambda t: text_splitting_paragraph(t, chunk_length=300))

# Compute stats
num_chunks_par = df["paragraph_chunks"].apply(len)
print(f"Average number of paragraph-based chunks per document: {num_chunks_par.mean():.2f}")
print(f"Total number of paragraph-based chunks: {num_chunks_par.sum()}")

# Example comparison
example_idx = 432 #Check out other examples
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]['title'])
print("Original length:", len(df.iloc[example_idx]['text']))
print(f"Character based chunks: {len(df.iloc[example_idx]['naive_chunks'])}")
print(f"Paragraph based chunks: {len(df.iloc[example_idx]['paragraph_chunks'])}")

print("\nParagraph chunk preview:\n")
for i, c in enumerate(df.iloc[example_idx]['paragraph_chunks'][:6]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")


Average number of paragraph-based chunks per document: 8.95
Total number of paragraph-based chunks: 8946

--- Example document ---
Title: John Ballantine (banker)
Original length: 12204
Character based chunks: 62
Paragraph based chunks: 50

Paragraph chunk preview:

Chunk 1:
John Ballantine (1743–1812), was a Scottish merchant and banker and one of the greatest friends, admirers and closest confidants of Robert Burns.
--------------------------------------------------------------------------------
Chunk 2:
  Significantly Ballantine gave the poet advice on the selection of poems for his First Kilmarnock Edition as well as being asked for his opinion on the bard's poems.

Life and character
John was born in Ayr to William Ballantine, a baillie in Ayr and his mother was Elizabeth Bowman.
--------------------------------------------------------------------------------
Chunk 3:
 He was a merchant and a Banker and in 1787 he became the Provost of Ayr, during which time he helped establish A

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:  
In the paragraph-aware chunking method above, we simply split Wikipedia text using the '.' delimiter to approximate sentence boundaries.  
Discuss whether this is an effective strategy for creating meaningful chunks in a RAG system.
Propose one or more improved chunking strategies that could better capture document structure — and you may include code snippets to justify or demonstrate your approach.  
<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>

Splitting text solely using the period delimiter is a suboptimal strategy for RAG systems primarily due to semantic fragmentation. A single sentence often lacks the necessary context to stand alone as a retrievable unit; for instance, a chunk containing only "He did it" is meaningless without the preceding nouns. Additionally, this naive approach introduces false positives by incorrectly splitting on abbreviations and disregards the natural document structure, fracturing the cohesive ideas that are usually grouped within paragraphs.

A more robust alternative is Recursive Character Splitting, which serves as the industry standard for preserving semantic integrity.  Instead of forcing splits at arbitrary punctuation, this method prioritizes a hierarchy of separators to keep related text together. It attempts to split first by double newlines (paragraphs), then single newlines, and finally sentences only when necessary. This ensures that the system preserves the largest possible semantic units before resorting to granular fragmentation, as illustrated conceptually below:

separators would be in ordr ["\n\n", "\n", ".", " "]


<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: Consider a scenario where you want to perform RAG on source code (e.g., Python files, Java classes) instead of natural language text. Would the chunking methods demonstrated above (character-based and sentence/paragraph based with boundaries) work effectively for code? Explain why or why not, and describe how you would approach chunking source code to maintain semantic coherence. What specific characteristics of code structure would you need to consider?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>


Standard text-based chunking strategies are ill-suited for source code because code relies on rigid syntactic structures rather than linguistic sentence boundaries. A primary failure mode is the functional role of punctuation; in programming, a period acts as an accessor operator (e.g., \texttt{object.method()}) rather than a stop. Naive splitting on these characters fractures method calls, leaving the retriever with syntactically broken segments that lack functional context. Furthermore, code derives meaning from hierarchical scoping, meaning that arbitrary character-based splitting often creates "orphaned blocks" where a function's logic is separated from its definition.

To maintain semantic coherence, chunking must respect the programming language's syntax, specifically preserving parent containers like classes or functions.  An effective approach involves parsing the code into an Abstract Syntax Tree (AST) or splitting based on significant delimiters such as indentation levels in Python or closing braces in Java. This ensures that a chunk captures a complete logical unit, such as a full function definition including its signature, body, and return statement—rather than an arbitrary slice of text that cannot be statically analyzed or understood in isolation.

<hr style="border:10px solid green"> </hr>
</font></h4>

In [11]:
# Saving chunks ---

# Flatten chunks into a new DataFrame
records = []
for _, row in df.iterrows():
    doc_id = row["id"]
    title = row["title"]
    url = row["url"]
    for i, chunk in enumerate(row["paragraph_chunks"]):
        records.append({
            "doc_id": doc_id,
            "title": title,
            "url": url,
            "chunk_id": f"{doc_id}_chunk_{i}",
            "chunk_text": chunk.strip()
        })

# Create the flattened chunks DataFrame
chunks_df = pd.DataFrame(records)
print(f"Total chunks: {len(chunks_df)}")
print(f"Average chunk length: {chunks_df['chunk_text'].apply(len).mean():.2f} characters\n")

# Show example
print("Example rows:")
display(chunks_df.head())

chunks_df.to_csv("wikipedia_chunks.csv", index=False)
print("Chunks saved to 'wikipedia_chunks.csv'")
print(f"Total chunks: {len(chunks_df)}")


Total chunks: 8946
Average chunk length: 244.93 characters

Example rows:


,doc_id,title,url,chunk_id,chunk_text
0,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_0,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_1,Early life \nShe was born on 15 February in Bo...
2,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_2,Indrani studied at Presbyterian Girls School ...
3,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_3,She met him during the production of his song ...
4,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_4,After that he studied A/L at the Royal Institu...


Chunks saved to 'wikipedia_chunks.csv'
Total chunks: 8946


## <b>Part II: Embedding</b>

After chunking our documents, the next step is to convert text chunks into vector representations (embeddings). These embeddings capture the semantic meaning of the text in a high-dimensional space, allowing us to measure similarity between chunks and queries mathematically.

We will use **sentence-transformers/all-MiniLM-L6-v2**, a compact and efficient embedding model that produces 384-dimensional embeddings for English text. This model offers a strong balance between performance and computational efficiency, making it well-suited for our RAG pipeline.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill in the <code>embed()</code> function to encode text chunks and generate normalized embeddings using <code>sentence-transformers/all-MiniLM-L6-v2</code>.  
Then, apply it to all documents in <code>chunks_df</code> and store the results.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [12]:
# --- 2.3 Embedding Generation ---
# Load model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"Loaded embedding model: {model_name}")


# --- Define embedding function ---
def embed(text_list, doc_type="document"):
    """
    Encodes a list of texts and returns normalized embeddings.
    """
    encoded = tokenizer(
        [f"search_{doc_type}: {t}" for t in text_list],
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoded)## FILL THE GAP: forward pass through the model to obtain hidden states
        token_embeddings = output.last_hidden_state## FILL THE GAP: extract the last hidden state from the output
        pooled = token_embeddings.mean(dim=1)## FILL THE GAP: aggregate token embeddings (e.g., by summing along sequence dimension)
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)## FILL THE GAP: apply L2 normalization along the embedding dimension
    return pooled.cpu()



tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2


In [13]:
# --- Test with one example ---
sample_text = ["Artificial intelligence is transforming the world."]
sample_emb = embed(sample_text)
print(f"Sample embedding shape: {sample_emb.shape}")

# --- Apply to all chunks ---
print(f"\nGenerating embeddings for {len(chunks_df)} chunks...")

emb_list = []
for i in tqdm(range(0, len(chunks_df), 32)):
    batch = chunks_df["chunk_text"].iloc[i:i+32].tolist()
    emb = embed(batch, doc_type="document")
    emb_list.append(emb)

chunk_embeddings = torch.cat(emb_list, dim=0).numpy()
chunks_df["embedding"] = list(chunk_embeddings)

print("\nEmbeddings generated and added to DataFrame.")
print(chunks_df[["chunk_id", "title", "embedding"]].head())

Sample embedding shape: torch.Size([1, 384])

Generating embeddings for 8946 chunks...


100%|██████████| 280/280 [00:12<00:00, 21.98it/s]



Embeddings generated and added to DataFrame.
           chunk_id           title  \
0  64741026_chunk_0  Indrani Perera   
1  64741026_chunk_1  Indrani Perera   
2  64741026_chunk_2  Indrani Perera   
3  64741026_chunk_3  Indrani Perera   
4  64741026_chunk_4  Indrani Perera   

                                           embedding  
0  [-0.0708645, -0.03150357, -0.0058141155, -0.02...  
1  [-0.012864295, -0.019443626, -0.028137444, 0.0...  
2  [-0.06029346, -0.036576908, 0.018930344, 0.026...  
3  [-0.07509218, -0.0029661183, -0.00598709, 0.03...  
4  [-0.037770636, -0.033015274, 0.01882598, 0.021...  


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: In most retrieval systems, embeddings are represented as fixed-size vectors. Can you think of a way to design embeddings that can flexibly adjust their size or level of detail while still preserving meaningful similarity between representations? How could such an approach benefit Retrieval-Augmented Generation (RAG) systems in practice, particularly for improving efficiency or adapting to different computational budgets?

<hr style="border:10px solid red"> </hr>
<i>Hint:</i> You can find the answer in the paper <a href="https://arxiv.org/pdf/2205.13147" target="_blank">Matryoshka Representation Learning</a>.
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>


A robust solution to this challenge is Matryoshka Representation Learning, which designs embeddings where information is encoded hierarchically, analogous to Russian nesting dolls. In this framework, the model is trained using a specialized multi-granular loss function that enforces a structural constraint: the initial dimensions (e.g., the first 64) must independently form a high-quality semantic representation.  Subsequent dimensions serve to progressively add granular detail to this foundational representation, rather than scattering information uniformly across the entire vector space.

For Retrieval-Augmented Generation systems, this approach offers significant efficiency gains by enabling adaptive retrieval strategies. A system can perform an initial, rapid approximate search using heavily truncated vectors to filter candidates with minimal computational cost, and then utilize the full-precision vectors only for the final re-ranking step. This allows the pipeline to dynamically scale between high speed and high accuracy depending on the available computational budget.

<hr style="border:10px solid green"> </hr>
</font></h4>

### **Building a Simple Vector Database**

After generating embeddings for all our document chunks, the next step is to **store** them in a structure that allows fast similarity search.  
In this section, we will build a **simple in-memory vector database** using PyTorch tensors.  
Each entry in the database will correspond to a text chunk and its embedding, enabling efficient retrieval based on vector similarity.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill in the code to populate the database by computing embeddings for all text chunks in <code>chunks_df</code> using the <code>embed()</code> function.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [14]:

def populate_database(chunks_df, batch_size=16):
    """
    Populates a vector database from precomputed chunks_df.
    """
    n_chunks = len(chunks_df)

    chunk_list = chunks_df["chunk_text"].tolist()

    sample_emb = embed([chunk_list[0]])## FILL THE GAP: compute one sample embedding to infer the output dimension
    output_dim = sample_emb.shape[1]## FILL THE GAP: extract the embedding dimension from the sample

    vectorial_database = torch.zeros((n_chunks, output_dim)) ## FILL THE GAP: initialize an empty tensor to store all embeddings


    print(f"Populating vector database with {n_chunks} chunks...")

    n = 0
    for i in range(0, n_chunks, batch_size):
        batch = chunk_list[i : i + batch_size]## FILL THE GAP: select a batch of chunk texts
        embeddings = embed(batch)## FILL THE GAP: compute embeddings for the current batch
        vectorial_database[n:n + len(batch)] = embeddings## FILL THE GAP: store embeddings in the tensor
        n += len(batch)

    return chunk_list, vectorial_database



In [15]:
#Build Vector Database
chunk_list, vectorial_database = populate_database(chunks_df)

print("\n✅ Vector database successfully built.")
print(f"Total stored chunks: {len(chunk_list)}")
print(f"Database tensor shape: {tuple(vectorial_database.shape)}")

Populating vector database with 8946 chunks...

✅ Vector database successfully built.
Total stored chunks: 8946
Database tensor shape: (8946, 384)


In [16]:
#Save vector databse
os.makedirs("vector_db", exist_ok=True)

# Save tensor + chunk list
torch.save(vectorial_database, "vector_db/vectorial_database.pth")

with open("vector_db/chunk_list.json", "w", encoding="utf-8") as f:
    json.dump(chunk_list, f, indent=4, ensure_ascii=False)

print("✅ Saved:")
print(" - vector_db/vectorial_database.pth")
print(" - vector_db/chunk_list.json")

✅ Saved:
 - vector_db/vectorial_database.pth
 - vector_db/chunk_list.json


In [17]:
# Load the database
vectorial_database = torch.load("vector_db/vectorial_database.pth", map_location=device)
vectorial_database.requires_grad_(False)

with open("vector_db/chunk_list.json", "r", encoding="utf-8") as f:
    chunk_list = json.load(f)

print(f"✅ Loaded {len(chunk_list)} chunks.")
print(f"Database shape: {vectorial_database.shape}\n")

# Inspect first few entries
for i, embedding_vector in enumerate(vectorial_database[:5]):
    print(f"Vector {i} → {embedding_vector[:5]}")
    print(f"Text snippet: {chunk_list[i][:300]}\n")


✅ Loaded 8946 chunks.
Database shape: torch.Size([8946, 384])

Vector 0 → tensor([-0.1097, -0.0133, -0.0467, -0.0494, -0.0300], device='cuda:0')
Text snippet: Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer. Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio".

Vector 1 → tensor([-0.0152, -0.0011, -0.1108,  0.0976, -0.0439], device='cuda:0')
Text snippet: Early life 
She was born on 15 February in Borella, and is the second of three girls in the family. Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian. She has one elder sister, Mallika and one younger sister, Iranganie.

Vector 2 → tensor([-0.0936, -0.0453, -0.0106, -0.0055, -0.0743], device='cuda:0')
Text snippet: Indrani studied at  Presbyterian Girls School in Regent Street. She studied Kandyan Dancing in the school. Her sister Mallika has been singing since 1965 where she was the playba

#### **Defining Similarity Metrics for Retrieval**

After populating our vector database with embeddings, the next step in a RAG pipeline is to define a *similarity metric* to measure how close two vectors are in the embedding space.  
Common metrics include **dot product**, **L2 distance**, and **cosine similarity**.  

In most retrieval systems, cosine similarity is preferred because it measures the *angle* between two vectors rather than their magnitude, allowing comparison based purely on semantic direction instead of scale.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill in the code to implement the <code>cosine_similarity()</code> function.  
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [18]:
def cosine_similarity(query_embeddings, doc_embeddings):
    """
    Computes cosine similarity between query and document embeddings
    using manual normalization.
    """
    query_magnitudes = query_embeddings.norm(dim=1, keepdim=True) ## FILL THE GAP: calculate vector length for each query
    normalized_queries = query_embeddings / query_magnitudes## FILL THE GAP: normalize queries using their magnitudes

    doc_magnitudes = doc_embeddings.norm(dim=1, keepdim=True)## FILL THE GAP: calculate vector length for each document
    normalized_docs = doc_embeddings / doc_magnitudes## FILL THE GAP: normalize documents using their magnitudes

    similarity_matrix = torch.mm(normalized_queries, normalized_docs.t())## FILL THE GAP: perform dot product between normalized queries and documents

    return similarity_matrix


# --- Example test ---
query_embeddings = embed([
    "What is t-SNE?",
    "Who is Laurens van der Maaten?"
], "query")

doc_embeddings = embed([
    "t-SNE is a dimensionality reduction algorithm created by Laurens van der Maaten."
], "document")

with torch.no_grad():
    sim_cos = cosine_similarity(query_embeddings, doc_embeddings)

print("🔍 Example cosine similarity scores:\n", sim_cos)

🔍 Example cosine similarity scores:
 tensor([[0.6605],
        [0.4333]])


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill in the code to complete the <code>retrieve()</code> function.  
It should encode the input query using <code>embed()</code>, compute similarity with all vectors in <code>vectorial_database</code>, and return the top-<i>k</i> most similar text chunks.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [19]:
def retrieve(query,
             vectorial_database=vectorial_database,
             chunk_list=chunk_list,
             topk=5,
             verbose=False):
    """
    Retrieves top-k most similar chunks to a query using dot-product similarity.
    """
    with torch.no_grad():
        query_embedding = embed([query], "query") ## FILL THE GAP: encode the input query using the embed() function
        query_embedding = query_embedding.to(vectorial_database.device)## FILL THE GAP: move the query embedding to the correct device
        similarity_scores = torch.mm(query_embedding, vectorial_database.t())## FILL THE GAP: compute similarity between query and database embeddings
        topk_results = torch.topk(similarity_scores, k=topk)## FILL THE GAP: extract the top-k highest similarity scores and indices

        if verbose:
            for score, idx in zip(topk_results.values[0], topk_results.indices[0]):
                print(f"\nScore: {score:.4f}")
                print(f"Chunk:\n{chunk_list[idx][:500]}\n{'-'*80}")

        retrieved_chunks = [chunk_list[idx] for idx in topk_results.indices[0].tolist()]## FILL THE GAP: select text chunks corresponding to the top-k indices
        return "\n".join(retrieved_chunks)## FILL THE GAP: return concatenated retrieved chunks as a single string


In [22]:
# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)



Score: 0.5877
Chunk:
Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016. Boria defeated former Miami-Dade School Board member Frank Bolaños in the 2012 elections, obtaining 54% of the vote.
--------------------------------------------------------------------------------

Score: 0.5618
Chunk:
(1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (1776–1813)
Sede vacante (1813–1817)
 Aloisio San Vitale (1817–1836)
 Giovanni Tommaso Neuschel (1836–1843)
 Pier Grisologo Basetti (1843–1857)
Sede vacante (16 June 1857 – 20 June 1859)
 Francesco Ben
--------------------------------------------------------------------------------

Score: 0.5258
Chunk:
22 see below)

Bishops

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 4: There are retrieval methods like BM25 that rely on lexical overlap between the query and documents, and others based on dense embeddings that capture semantic similarity beyond exact word matches. Explain how these two approaches differ in how they represent and compare text. Then, discuss how a hybrid retrieval strategy combining both can overcome their respective limitations and improve retrieval performance in RAG systems. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>

BM25 and dense embeddings utilize fundamentally different representations of text: BM25 relies on sparse, lexical matching to prioritize exact keyword overlap, while dense embeddings project text into a continuous vector space to measure semantic similarity based on meaning rather than specific vocabulary.  A hybrid strategy leverages the strengths of both by running these retrievers in parallel and fusing their rankings—often via methods like Reciprocal Rank Fusion—thereby allowing a RAG system to simultaneously capture broad conceptual relevance via embeddings without losing the ability to pinpoint specific entities, acronyms, or technical terms that purely semantic models might abstract away.
<hr style="border:10px solid green"> </hr>
</font></h4>


In real-world RAG systems, instead of manually storing and comparing vectors, we rely on **vector databases** such as **ChromaDB**, which are optimized for efficient **similarity search**, **indexing**, and **retrieval** at scale.  

These databases provide:
- Fast nearest-neighbor search (e.g., using HNSW graphs)  
- Persistent storage for millions of embeddings  
- Built-in support for different similarity metrics (cosine, L2, inner product)  


In [20]:

import chromadb
from chromadb.config import Settings

# --- Initialize ChromaDB client ---
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset ensures a clean state
chroma_client.reset()

# --- Create a collection ---
# You can choose the similarity metric: "cosine", "l2", or "ip" (inner product)
collection_name = "wikipedia_chunks"
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine similarity works best for normalized embeddings
)

print(f"✅ Created collection: {collection_name}")


✅ Created collection: wikipedia_chunks


In [22]:
# Prepare the data
chunks_df = chunks_df.head(5461)
ids = chunks_df["chunk_id"].tolist()
embeddings = chunks_df["embedding"].tolist()
documents = chunks_df["chunk_text"].tolist()

# Ensure all embeddings are plain Python lists
embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]

# Add useful metadata for inspection
metadatas = [
    {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "url": row["url"]
    }
    for _, row in chunks_df.iterrows()
]

# Add to the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

print(f"\n✅ Added {collection.count()} chunks to the collection")
print(f"Collection metadata: {collection.metadata}")



✅ Added 5461 chunks to the collection
Collection metadata: {'hnsw:space': 'cosine'}


In [23]:
# Encode the query using our embed() function
query = "When was Luigi Boria born?"
query_embedding = embed([query], doc_type="query")[0].tolist()  # get single vector as list

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3 # number of retrieved results
)

# Display results
print(f"🔎 Query: {query}\n" + "=" * 80)
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"Title: {results['metadatas'][0][i]['title']}")
    print(f"Similarity score: {1 - results['distances'][0][i]:.4f}")  # cosine distance → similarity
    print(f"Text: {results['documents'][0][i][:300]}...")
    print("-" * 80)


🔎 Query: When was Luigi Boria born?

Result 1:
Title: Matías Succar
Similarity score: 0.4618
Text: He then moved to changed position to left-back, and then a left midfielder, before joining Cantolao San Borja. Aged 14, he moved to Academia Cantolao where he became a striker. He didn't play much, and left football for one year....
--------------------------------------------------------------------------------

Result 2:
Title: Roman Catholic Diocese of Pistoia
Similarity score: 0.4432
Text: (1426–1434 Died)
Donato de' Medici (1436–1474)
Niccolò Pandolfini (1474–1518)

from 1500 to 1652

Cardinal Lorenzo Pucci (1518)
Cardinal Antonio Pucci 1518–1541 Resigned)
Cardinal Roberto Pucci (1541–1546)
Francesco da Galliano (1546–1559 Died)
Giovambattista Ricasoli (1560–1572 Died)
Alessandro Ott...
--------------------------------------------------------------------------------

Result 3:
Title: Luca Baldisserri
Similarity score: 0.3817
Text: Luca Baldisserri (born 11 December 1962) is an Italia

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5: </b><br>
ChromaDB and other vector databases often rely on Hierarchical Navigable Small World (HNSW) graphs to perform efficient approximate nearest neighbor search.  
Explain how the HNSW algorithm organizes data to enable fast and accurate retrieval in high-dimensional spaces.  
Why is this structure particularly effective for large-scale embedding collections compared to brute-force search?

<hr style="border:10px solid red"> </hr>
<i>Reference:</i> <a href="https://arxiv.org/pdf/1603.09320" target="_blank">Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs</a>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>

HNSW organizes data into a hierarchical multi-layered graph structure, where upper layers act as sparse "expressways" for rapid traversal across the vector space and lower layers provide dense connectivity for precise local search.  By utilizing a greedy search mechanism that transitions from coarse to fine granularity, the algorithm achieves a logarithmic time complexity of $O(\log N)$, a significant improvement over the linear $O(N)$ cost required by brute-force methods. This architecture enables the system to bypass the vast majority of irrelevant data points, thereby allowing for millisecond-latency retrieval on massive datasets with only a negligible trade-off in accuracy.

<hr style="border:10px solid green"> </hr>
</font></h4>


### <b> Two-Stage Retrieval: Dense Retrieval + Reranker</b>

In RAG, the first retrieval step often returns passages that are similar in meaning but not always the most relevant.  
A **reranker** fixes this by re-evaluating the top retrieved chunks using a stronger model that jointly reads the query and each document to assign a more accurate relevance score.


In [24]:
from sentence_transformers import CrossEncoder

# Load reranker model
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL_NAME)

def retrieve_with_reranker(query, collection, initial_k=5, final_k=3):
    """
    Retrieves and reranks candidate documents for a given query.
    Returns both the initial dense results and reranked results.
    """
    query_embedding = embed([query], doc_type="query")[0].tolist()
    candidates = collection.query(query_embeddings=[query_embedding], n_results=initial_k)

    docs = candidates["documents"][0]
    metas = candidates["metadatas"][0]
    dense_scores = [(1 - s) for s in candidates["distances"][0]]

    pairs = [(query, d) for d in docs]
    ce_scores = reranker.predict(pairs)

    reranked = [
        {
            "title": metas[i].get("title", ""),
            "url": metas[i].get("url", ""),
            "text": docs[i],
            "dense_score": dense_scores[i],
            "rerank_score": float(ce_scores[i]),
        }
        for i in range(len(docs))
    ]
    reranked.sort(key=lambda x: x["rerank_score"], reverse=True)

    return docs, metas, dense_scores, reranked[:final_k]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [25]:
# Example usage
query = "Where did Luigi Boria study?" #Try other queries too
docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

# --- Display initial dense retrieval ---
print(f"\nInitial dense retrieval (Top 5):\n" + "=" * 80)
for i, (d, s, m) in enumerate(zip(docs, dense_scores, metas), 1):
    print(f"{i}. {m.get('title', '')}  |  Dense similarity: {s:.4f}")
    print(f"Text: {d[:300].replace('\n', ' ')}")
    print("-" * 80)

# --- Display top reranked results ---
print(f"\nAfter Cross-Encoder Reranking (Top 3):\n" + "=" * 80)
for i, item in enumerate(top_reranked, 1):
    print(f"{i}. {item['title']}")
    print(f"Dense similarity: {item['dense_score']:.4f} | Reranker score: {item['rerank_score']:.4f}")
    print(f"Text: {item['text'][:300].replace('\n', ' ')}")
    print("-" * 80)


Initial dense retrieval (Top 5):
1. Matías Succar  |  Dense similarity: 0.4488
Text: He then moved to changed position to left-back, and then a left midfielder, before joining Cantolao San Borja. Aged 14, he moved to Academia Cantolao where he became a striker. He didn't play much, and left football for one year.
--------------------------------------------------------------------------------
2. Émile Digneffe  |  Dense similarity: 0.4029
Text: Les hommes, les entreprises, les reseaux, Brussel, De Boeck Université, 1996, p. 242–243. D'Hoore, Marc, in : Delforge, Paul, Destatte Philippe, Libon, Micheline (ed.), Encyclopédie du Mouvement wallon, Charleroi, Institut Jules Destrée, 2001, 3 dl., p. 502–503.
--------------------------------------------------------------------------------
3. Paolo Sassone-Corsi  |  Dense similarity: 0.3927
Text: In search of innovation, he and his wife moved to San Diego, United States where he worked as a post-doctoral fellow under Inder Verma at the Salk I

![Bi-Encoder vs Cross-Encoder Architecture](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6:  
The figure above compares a Bi-Encoder and a Cross-Encoder architecture.  
Rerankers such as <code>cross-encoder/ms-marco-MiniLM-L-6-v2</code> use the second approach, jointly encoding the query and document through a single transformer.  
Why does this joint encoding typically yield higher retrieval precision, and why is it applied as a second-stage reranker instead of being used directly for large-scale retrieval?
<hr style="border:10px solid red"> </hr>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>

Cross-Encoders achieve superior precision by processing the query and document simultaneously within a single transformer pass. Unlike Bi-Encoders, which compress inputs into independent vector representations, the Cross-Encoder architecture enables the self-attention mechanism to model deep, token-level dependencies between the query and the text. This allows the model to capture subtle nuances and semantic relationships that are often lost during separate vector compression.

However, this accuracy comes at a significant computational cost. Because the output depends on the specific pairing of inputs, document representations cannot be pre-computed or indexed. Consequently, the system must execute a full Transformer forward pass for every potential match, resulting in linear computational complexity $\mathcal{O}(N)$ relative to the corpus size. This makes Cross-Encoders prohibitively expensive for initial large-scale retrieval, restricting their practical utility to second-stage reranking where they process only a small, pre-filtered set of candidates.

<hr style="border:10px solid green"> </hr>
</font></h4>


### **Integrating Retrieved Context into the LLM’s Prompt**

Now that we can retrieve and rerank the most relevant document chunks,  
we integrate them directly into the **language model’s prompt**.  
This step allows the model to **ground its answer on factual context** rather than relying solely on internal knowledge —  
thereby improving accuracy and reducing hallucinations.


In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
gen_tok = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

def generate(msg, max_new_tokens=128, temperature=0.2):
    messages = [{"role": "user", "content": msg}]
    inputs = gen_tok.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    outputs = gen_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        eos_token_id=gen_tok.eos_token_id,
        pad_token_id=gen_tok.pad_token_id if gen_tok.pad_token_id is not None else gen_tok.eos_token_id
    )

    return gen_tok.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()


# ============================================================
# NO-RAG vs WITH RAG
# ============================================================
query = "When was Luigi Boria born?" #Try other queries

print("ORIGINAL PROMPT\n" + "=" * 60)
print(query)

print("\nANSWER WITHOUT RAG\n" + "=" * 60)
print(generate(query))

docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

context = "\n\n".join(h["text"] for h in top_reranked)[:1600]
rag_prompt = (
    f"Use only the context to answer. If unknown, say you don't know.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\nAnswer:"
)

print("\nAUGMENTED PROMPT\n" + "=" * 60)
print(rag_prompt)

print("\nANSWER WITH RAG\n" + "=" * 60)
print(generate(rag_prompt))


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

ORIGINAL PROMPT
When was Luigi Boria born?

ANSWER WITHOUT RAG
Luigi Boria was born on November 10, 1953. He is an Italian-American actor and director known for his work in films such as "The Godfather" (1972), "The Godfather: Part II" (1974), and "The Godfather: Part III" (1980). His career has spanned over four decades, during which he has received numerous awards and nominations for his performances.

AUGMENTED PROMPT
Use only the context to answer. If unknown, say you don't know.

Context:
Luca Baldisserri (born 11 December 1962) is an Italian engineer, best known for being the Chief Track Engineer for Scuderia Ferrari between 2007 and 2015.

Career

Scuderia Ferrari
He joined Scuderia Ferrari in 1989, and in 1995 he became Gerhard Berger's race engineer.

He then moved to changed position to left-back, and then a left midfielder, before joining Cantolao San Borja. Aged 14, he moved to Academia Cantolao where he became a striker. He didn't play much, and left football for one year.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 7: In the RAG prompt construction step, we simply concatenate the retrieved chunks before the question. Discuss potential issues with this naive approach, such as token limits, redundancy, or irrelevant context dilution. Then, explain how we could select, weight, or summarize the retrieved chunks before injecting them into the prompt to improve generation quality and efficiency. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>

Naive concatenation of retrieved chunks frequently precipitates context window overflows and exacerbates the "Lost in the Middle" phenomenon, where the generation model fails to attend to critical information buried amidst redundant or irrelevant noise.  This approach often dilutes the semantic signal, leading to higher latency and costs without a corresponding increase in response accuracy. To mitigate these issues, systems should employ sophisticated filtering strategies, such as re-ranking with Cross-Encoders, to select only the most distinct and high-utility chunks, or utilize extractive summarization to compress the context into a dense representation prior to injection. Furthermore, strategically placing the most relevant information at the beginning and end of the prompt can counteract attention drift, ensuring higher fidelity in the generated response by leveraging the model's U-shaped attention capabilities.

<hr style="border:10px solid green"> </hr>
</font></h4>


#### **To go further**

- Experiment with other chunking methods (e.g., semantic or recursive chunking).  
- Explore **LangChain** and **LlamaIndex** for building modular RAG pipelines.  
- Try **hybrid retrieval** combining sparse (BM25) and dense embeddings.  
- Explore more advanced RAG methods such as **RAG-Fusion**, **Self-RAG**, and **Active-RAG**.  
- Experiment with **Matryoshka Representation Learning** for scalable embeddings.  
- Try **fine-tuning rerankers** or **retrievers** for domain-specific data.  
